In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
data_path = '../data/'
listing_info_fn = 'listing_info.csv'
test_fn = 'test.csv'
train_fn = 'train.csv'
user_behavior_logs_fn = 'user_behavior_logs.csv'
user_info_fn = 'user_info.csv'
user_repay_logs_fn  = 'user_repay_logs.csv'
user_taglist_fn = 'user_taglist.csv'

In [ ]:
preprocess_listing_info_fn = 'preprocess_listing_info.csv'
preprocess_test_fn = 'preprocess_test.csv'
preprocess_train_fn = 'preprocess_train.csv'
preprocess_user_behavior_logs_fn = 'preprocess_user_behavior_logs.csv'
preprocess_user_info_fn = 'preprocess_user_info.csv'
preprocess_user_repay_logs_fn  = 'preprocess_user_repay_logs.csv'
preprocess_user_taglist_fn = 'preprocess_user_taglist.csv'

In [ ]:
preprocess_train_data_listing_info_fn = 'preprocess_train_data_listing_info.csv'

### listing_info

In [ ]:
listing_info_data = pd.read_csv(data_path+listing_info_fn)

In [ ]:
listing_info_data.loc[listing_info_data['user_id'] == 712075].sort_values('auditing_date')

In [ ]:
listing_info_data.isnull().sum()

In [ ]:
listing_info_data.columns

### preprocess train

In [ ]:
train_data = pd.read_csv(data_path+train_fn)

In [ ]:
# due_interval
train_data['due_interval'] = (pd.to_datetime(train_data['due_date']) - pd.to_datetime(train_data['auditing_date'])).dt.days

In [ ]:
train_data.head()

In [ ]:
train_data_2 = train_data.loc[train_data['repay_date']==r'\N']
train_data_1 = train_data.loc[train_data['repay_date']!=r'\N']
train_data_1.loc[:,'repay_interval'] = (pd.to_datetime(train_data_1['repay_date']) - pd.to_datetime(train_data_1['auditing_date'])).dt.days
train_data_2.loc[:,'repay_interval'] = 60000
train_data = train_data_1.append(train_data_2)
train_data = train_data.sort_index()

In [ ]:
train_data.head(10)

In [ ]:
train_data['repay_due_interval'] = train_data['repay_interval'] - train_data['due_interval']
train_data.loc[train_data['repay_due_interval'] > 0, 'repay_due_interval'] = 1
train_data.drop(['repay_interval'], axis=1, inplace=True)
train_data['repay_amt'] = train_data['due_amt']
train_data.loc[train_data['repay_due_interval'] > 0, 'repay_amt'] = 0

# train_data.to_csv ("train_preprocess.csv", index=False, encoding = "utf-8")
# df = pd.read_csv("./train_preprocess.csv", encoding = "utf-8")
# df.head()

In [ ]:
train_data.head()

In [ ]:
train_data.to_csv(data_path+preprocess_train_fn, index=False, encoding = "utf-8")
df = pd.read_csv(data_path+preprocess_train_fn, encoding = "utf-8")
df.head()

### preprocess user_info

In [ ]:
user_info = pd.read_csv(data_path+user_info_fn, header=0, index_col=False, names=['user_id', 'reg_mon', 'gender', 'age', 'cell_province', 'id_province', 'id_city', 'insertdate'])
gender_mapping = {label: idx for idx,label in enumerate(set(user_info['gender']))}
user_info['gender'] = user_info['gender'].map(gender_mapping)

In [ ]:
cell_province_mapping = {label: idx for idx,label in enumerate(sorted(set(user_info['cell_province']), reverse=True))}
# print('\ncell_province_mapping:', cell_province_mapping)
user_info['cell_province'] = user_info['cell_province'].map(cell_province_mapping)

id_province_mapping = {label: idx for idx,label in enumerate(sorted(set(user_info['id_province']), reverse=True))}
# print('\nid_province_mapping:', id_province_mapping)
user_info['id_province'] = user_info['id_province'].map(id_province_mapping)

id_city_mapping = {label: idx for idx,label in enumerate(sorted(set(user_info['id_city']), reverse=True))}
# print('\nid_city_mapping:', id_city_mapping)
user_info['id_city'] = user_info['id_city'].map(id_city_mapping)

user_info['insert_ts'] = user_info['insertdate'].apply(lambda date: pd.to_datetime(date).timestamp())

user_info.head()

In [ ]:
user_info = pd.read_csv(data_path+preprocess_user_info_fn)
user_info.drop(['Unnamed: 0'], axis=1,inplace=True)

In [ ]:
import copy
user_info = pd.concat([user_info, pd.DataFrame(columns=['insertdata_first','insertdata_last', 'insertdata_times'])], sort=True)
user_info_copy = user_info.copy(deep=True)

In [ ]:
handled_index_list = []
count = 0
for idx, row in user_info.iterrows():
    count += 1
    if count%100 == 0:
        print(count)
    try:
        temp_user = user_info_copy.loc[user_info_copy['user_id'] == row['user_id']]
        temp_user_insterdata_times = len(temp_user)
        if temp_user_insterdata_times == 0:
            print('error')
            break
        if temp_user_insterdata_times == 1:        
            user_info_copy.loc[idx, 'insertdata_first'] = temp_user['insert_ts'].values[0]
            user_info_copy.loc[idx, 'insertdata_last'] = temp_user['insert_ts'].values[0]
            user_info_copy.loc[idx, 'insertdata_times'] = 1
            continue
        if idx in handled_index_list:
            continue
        else:
            handled_index_list.append(idx)
        temp_user_ts = temp_user['insert_ts']
        temp_user_ts_max = temp_user_ts.max()
        temp_user_ts_min = temp_user_ts.min()
        temp_user_ts_dict = {v:k for k, v in temp_user_ts.to_dict().items()}
        temp_user_ts_max_idx = temp_user_ts_dict[temp_user_ts_max]
        temp_user_index_list = temp_user.index.to_list()
        temp_user_index_list.remove(temp_user_ts_max_idx)
        user_info_copy.drop(index=temp_user_index_list, axis=0, inplace=True)
        user_info_copy.loc[temp_user_ts_max_idx, 'insertdata_first'] = temp_user_ts_min
        user_info_copy.loc[temp_user_ts_max_idx, 'insertdata_last'] = temp_user_ts_max
        user_info_copy.loc[temp_user_ts_max_idx, 'insertdata_times'] = temp_user_insterdata_times
    except:
        print(row['user_id'])
        break

In [ ]:
user_info_copy.loc[user_info_copy['user_id'] == 173388.0]

In [ ]:
user_ts_670044_ts = user_info.loc[user_info['user_id'] == 670044]['insert_ts']
user_ts_670044_dict = {v:k for k, v in user_ts_670044_ts.to_dict().items()}
print('ts:', user_ts_670044_ts.min(), user_ts_670044_ts.max())
print('index:', user_ts_670044_dict[user_ts_670044_ts.min()], user_ts_670044_dict[user_ts_670044_ts.max()])

In [ ]:
user_ts_670044 = user_info.loc[user_info['user_id'] == 670044]
handled_index_list = []
print(user_ts_670044.index.to_list() in [[59861, 61420, 61588]])
handled_index_list.append(user_ts_670044.index.to_list())
handled_index_list
user_ts_670044_index_list = user_ts_670044.index.to_list()
user_ts_670044_index_list.remove(user_ts_670044_dict[user_ts_670044_ts.max()])
print(user_ts_670044_index_list)

In [ ]:
user_ts_670044_dict[user_ts_670044_ts.max()]

In [ ]:
print(user_ts_670044.index.to_list())

In [ ]:
user_ts_670044_dict[user_ts_670044_ts.max()]

In [ ]:
print([1,2,3].remove(1))

In [ ]:
user_info.to_csv(data_path+preprocess_user_info_fn, index=False, encoding='utf-8')
df = pd.read_csv(data_path+preprocess_user_info_fn, encoding='utf-8')
df.head()

### preprocess user_repay_logs

In [ ]:
user_repay_logs_data = pd.read_csv(data_path+user_repay_logs_fn)
user_repay_logs_data.head()

In [ ]:
user_repay_logs_data.loc[user_repay_logs_data['user_id'] == 777528].sort_values(['listing_id', 'due_date']).head()

In [ ]:
user_repay_logs_data.loc[user_repay_logs_data['repay_date'] == r'2200-01-01'].head()

In [ ]:
user_repay_logs_data['repay_due_interval'] = (pd.to_datetime(user_repay_logs_data['repay_date']) - pd.to_datetime(user_repay_logs_data['due_date'])).dt.days
user_repay_logs_data.head()

In [ ]:
user_repay_logs_data.to_csv(data_path+preprocess_user_repay_logs_fn)
df = pd.read_csv(data_path+preprocess_user_repay_logs_fn, encoding = "utf-8")
df.head()

### preprocess user_taglist

In [ ]:
user_taglist_data = pd.read_csv(data_path+user_taglist_fn)
user_taglist_data.head()

In [ ]:
user_taglist_new = user_taglist_data.drop(['taglist'], axis=1).join(user_taglist_data['taglist'].str.split('|', expand=True).stack().reset_index(level=1, drop=True).rename('tag'))
user_taglist_new.head()

In [ ]:
user_taglist_new.to_csv(data_path+preprocess_user_taglist_fn, index=False, encoding = "utf-8")
del user_taglist_new

### Contact

#### (1) listing_info & train

In [ ]:
train_data = pd.read_csv(data_path+preprocess_train_fn)
listing_info_data = pd.read_csv(data_path+listing_info_fn, header=0, index_col=False, names=['user_id', 'listing_id', 'auditing_date', 'term', 'rate', 'principal'])
train_listing_info_data = pd.merge(train_data, listing_info_data, on=['user_id', 'listing_id', 'auditing_date'])
user_info_data = pd.read_csv(data_path+preprocess_user_info_fn)

In [ ]:
print(train_listing_info_data.shape)
train_listing_info_data.head()

In [ ]:
train_listing_info_data['auditing_ts'] = train_listing_info_data['auditing_date'].apply(lambda date: pd.to_datetime(date).timestamp())

In [ ]:
train_listing_info_data.head()

In [ ]:
train_listing_info_data['due_ts'] = train_listing_info_data['due_date'].apply(lambda date: pd.to_datetime(date).timestamp())
train_listing_info_data.head()

In [ ]:
train_listing_info_data.to_csv(data_path+preprocess_train_data_listing_info_fn, index=False, encoding='utf-8')
del train_listing_info_data

#### (2) user_info

In [ ]:
train_listing_info_data = pd.read_csv(data_path+preprocess_train_data_listing_info_fn, encoding='utf-8')
user_info_data = pd.read_csv(data_path+preprocess_user_info_fn)
user_info_data.drop(['Unnamed: 0'], axis=1,inplace=True)

In [ ]:
count = 0
new_train_listing_info_data = None
for index,row in train_listing_info_data.iterrows():
    count += 1
    if count%100 == 0:
        print(count)
#         break
    temp_trainlist_rdf = row.to_frame().T.rename({0:index}, axis='index')
#     print('temp_trainlist_rdf:', temp_trainlist_rdf)
    valid_ts = row['auditing_ts']
    uid = row['user_id']
    matchinfo = user_info_data.loc[user_info_data['user_id']==uid]
    if len(matchinfo) > 1:
        matchinfo = matchinfo.sort_values('insert_ts', ascending=False)
        for i,r in matchinfo.iterrows():
            if r['insert_ts'] < valid_ts:
#                 print('temp_trainlist_rdf:', temp_trainlist_rdf)
                temp_user_rdf = r.to_frame().T.rename({i:index}, axis='index')
                temp_trainlist_rdf = pd.merge(temp_trainlist_rdf, temp_user_rdf, left_on=['user_id'], right_on=['user_id'], how='outer', sort=True).rename({0:index}, axis='index')
#                 print('temp_user_rdf:', temp_user_rdf)
#                 print('temp_trainlist_rdf', temp_trainlist_rdf)
#                 print('temp_trainlist_rdf:', temp_trainlist_rdf)
                if new_train_listing_info_data is None:
                    new_train_listing_info_data = temp_trainlist_rdf
                else:
                    new_train_listing_info_data = pd.concat([new_train_listing_info_data, temp_trainlist_rdf], sort=True)
        
        print('new_train_listing_info_data:',new_train_listing_info_data)
        if new_train_listing_info_data is not None:
            if 'age_x' in new_train_listing_info_data.columns:
                break
#                 print('===', new_train_listing_info_data)
#                 train_listing_info_data = pd.merge(train_listing_info_data, rdf, on=['user_id'], how='outer')
#         print(train_listing_info_data.loc[train_listing_info_data['user_id'] == 795130])
#         break

In [ ]:
train_listing_info_data.loc[train_listing_info_data['user_id'] == 957]

In [ ]:
new_train_listing_info_data.loc[957,:]

In [ ]:
user_info_data.loc[user_info_data['user_id'] == 595855]

In [ ]:
new_train_listing_info_data.head()

In [ ]:
new_train_listing_info_data.shape

In [ ]:
new_train_listing_info_data.columns

In [ ]:
train_listing_info_data.loc[train_listing_info_data['user_id'] == 795130]

#### (3) data

In [ ]:
train_data = pd.read_csv('train_preprocess.csv')
listing_info = pd.read_csv('listing_info.csv', index_col=False, names=['user_id', 'listing_id', 'auditing_date', 'term', 'rate', 'principal'])
user_info = pd.read_csv('user_info_preprocess.csv')

In [ ]:
train_data_listing_info = pd.merge(train_data, listing_info, on=['user_id', 'listing_id', 'auditing_date'])
# train_data_listing_info.drop(['due_date'], axis=1, inplace=True)
train_data_listing_info['auditing_ts'] = train_data_listing_info['auditing_date'].apply(lambda date: pd.to_datetime(date).timestamp())
# train_data_listing_info['due_ts'] = train_data_listing_info['due_date'].apply(lambda date: pd.to_datetime(date).timestamp())
# train_data_listing_info['repay_ts'] = train_data_listing_info['repay_date'].apply(lambda date: pd.to_datetime(date).timestamp())
train_data_listing_info.head()